In [ ]:
'''
1. Número de estudiantes asociados/egresados/titulados en área de la agricultura de los últimos 20 años
2. Mapeo de agricultores, ¿Qué se está cultivando y en qué zonas?
3. Base de datos asociadas a agricultura
4. Montos de proyectos asociados a agricultura
5. Catastro frutícola, ¿Cuántas hectáreas de la macrozona centro sur se han establecido con cultivos frutícolas en los últimos 20 años?
6. Datos del último censo agrícola
'''
# De datos extraidos desde mineduc titulados podemos extraer información relevante al área, sin embargo los datos incluidos
# en mineduc comprenden desde el año 2007 hasta 2019 dispnibles en:
# https://bit.ly/2DnDKIV

import pandas as pd
import altair as alt

# Para pregunta 1
urls = ['http://datos.mineduc.cl/datasets/193402-titulados-de-educacion-superior-2019.download/',
        'http://datos.mineduc.cl/datasets/190756-titulados-de-educacion-superior-2018.download/',
        'http://datos.mineduc.cl/datasets/185829-titulados-de-educacion-superior-2017.download/',
        'http://datos.mineduc.cl/datasets/179074-titulados-de-educacion-superior-2016.download/',
        'http://datos.mineduc.cl/datasets/179073-titulados-de-educacion-superior-2015.download/',
        'http://datos.mineduc.cl/datasets/179072-titulados-de-educacion-superior-2014.download/',
        'http://datos.mineduc.cl/datasets/179071-titulados-de-educacion-superior-2013.download/',
        'http://datos.mineduc.cl/datasets/179070-titulados-de-educacion-superior-2012.download/',
        'http://datos.mineduc.cl/datasets/179069-titulados-de-educacion-superior-2011.download/',
        'http://datos.mineduc.cl/datasets/179068-titulados-de-educacion-superior-2010.download/',
        'http://datos.mineduc.cl/datasets/179067-titulados-de-educacion-superior-2009.download/',
        'http://datos.mineduc.cl/datasets/179066-titulados-de-educacion-superior-2008.download/',
        'http://datos.mineduc.cl/datasets/179065-titulados-de-educacion-superior-2007.download/']

dataframes = []
for url in urls:
    df_aux = pd.read_csv(url, sep=';', low_memory=False)
    #filtro macrozona
    df_aux = df_aux[df_aux['REGION_SEDE'].isin(['BIOBÍO','ÑUBLE',"LIB. GRAL B. O'HIGGINS",'MAULE'])]
    #filtro área
    df_aux = df_aux[df_aux['OECD_AREA']=='AGRICULTURA']
    dataframes.append(df_aux)

df_titulados = pd.concat(dataframes)

# Para pregunta 4
# Adjudicaciones de proyectos con fuente de ANID
URL = '../data/ANID/BDH_Proyectos.csv'
df_adjudicaciones = pd.read_csv(URL, sep=';', encoding='latin-1')

In [ ]:
df_titulados.columns

In [ ]:
df_titulados['AREA_CARRERA_GENERICA_N'].unique()

In [ ]:
df_titulados.head()

# Respuesta pregunta 1

In [ ]:
df_1 = df_titulados.groupby(['CAT_PERIODO']).agg({'CODIGO_UNICO': 'count'}).copy()
df_1 = df_1.reset_index().copy()
df_1["CAT_PERIODO"] = df_1["CAT_PERIODO"].astype(str).str[4:8]

df_1.head()

In [ ]:
#Gráfico
chart1 =  alt.Chart(df_1).mark_bar(color='darkseagreen').encode(
            x = alt.X('CAT_PERIODO',title=None),
            y = alt.Y('CODIGO_UNICO',title='N° Titulados'),
          )
text =  chart1.mark_text(color='forestgreen', fontSize = 13,
                         align='center', baseline='middle', dy=-10).encode(
          text = 'CODIGO_UNICO'
        )
(chart1+text).properties(width=500, height=200,
    title=['Titulados área Agricultura','Macrozona Centro-Sur','']
).configure_legend(titleFontSize=14,labelFontSize=10).configure_title(
    fontSize=20, font='Arial', anchor='middle',color='black'
  ).configure_axis(labelFontSize=13, titleFontSize=13)

# Respuesta pregunta 4


In [ ]:
# Para responder la pregunta tomamos como referencia los datos abiertos que provee la ANID en base a proyectos adjudicados al 2019
# Donde además podemos ver que Ñuble no aparece con ninguna adjudicación en este ámbito (ciencias agricolas)

df_4 = df_adjudicaciones[df_adjudicaciones['MACROZONA MINCIENCIA (segUn ax)'] == 'CENTRO SUR'].copy()
df_4 = df_4[df_4['SUBDIRECCION'].\
            isin(['PROYECTOS DE INVESTIGACION','INVESTIGACION APLICADA'])]
df_4 = df_4[df_4['AREA_OCDE'] == 'CIENCIAS AGRICOLAS']
df_4['REGION_EJECUCION'] = df_4['REGION_EJECUCION'].str.replace("'","")
df_4['MONTO ADJUDICADO (MILES $)'] = df_4['MONTO ADJUDICADO (MILES $)'].str.replace(',','.').astype(float)
df_4.head(3)

In [ ]:
# de los datos anteriores podemos obtener varias visualizaciones, una de ellas es el monto por región de la macrozona
# De ser necesario se puede incluir una comparación con la región metropolitana

df_4a = df_4.groupby(['REGION_EJECUCION']).agg({'MONTO ADJUDICADO (MILES $)': 'sum'}).copy()
df_4a = df_4a.reset_index().copy()

df_4a.head()


In [ ]:
# Otra opción es separar por tipo programa conicyt al que pertenece
df_4b = df_4.groupby(['PROGRAMA_CONICYT']).agg({'MONTO ADJUDICADO (MILES $)': 'sum'}).copy()
df_4b = df_4b.reset_index().copy()

df_4b.head()

In [ ]:
# Incluso se podría saber que instituciones se adjudican mayor cantidad de dinero
df_4c = df_4.groupby(['INSTITUCION_PRINCIPAL']).agg({'MONTO ADJUDICADO (MILES $)': 'sum'}).copy()
df_4c = df_4c.reset_index().copy()

df_4c.head(7)

In [ ]:
# El gráfico que planeamos es un híbrido entre la propuesta 1 y 2 
df_4g = df_4.groupby(['REGION_EJECUCION','PROGRAMA_CONICYT']).\
                      agg({'MONTO ADJUDICADO (MILES $)': 'sum'}).copy()

df_4g = df_4g.reset_index().rename(columns={'PROGRAMA_CONICYT': 'Programa CONICYT'})

df_4g.head()

In [ ]:
chart4 = alt.Chart(df_4g).mark_bar().encode(
          x = alt.X('MONTO ADJUDICADO (MILES $)'),
          y = alt.Y('REGION_EJECUCION',title=None),
          color='Programa CONICYT'
      )

text4 = alt.Chart(df_4g).mark_text(dx=42, color='black',fontSize=15).encode(
    x = alt.X('MONTO ADJUDICADO (MILES $)',stack='zero'),
    y = alt.Y('REGION_EJECUCION'),
    detail='Programa CONICYT',
    text=alt.Text('MONTO ADJUDICADO (MILES $)', format='.2f')
)
(chart4+text4).configure_range(
    category={'scheme': 'set2'}
).properties(width=450, height=100,
      title=['Montos adjudicados proyectos del área Ciencias Agrícolas', 'Macro zona Centro-Sur',' '])\
      .configure_title( fontSize=20, font='Arial', anchor='middle', color='black')\
      .configure_legend(titleFontSize=14,labelFontSize=13,strokeColor='gray',
                         fillColor='#EEEEEE', padding=10, cornerRadius=10)\
      .configure_axis(labelFontSize=13, titleFontSize=13)


# Respuesta Pregunta 6

In [ ]:
#estoy buscando el archivo del censo agricola, para ver si al menos éste es público
# El instituto nacional de estádistica ofrece datos sobre el área en particular por subsegmento/área (no sé como decirle)
# disponible en: https://www.ine.cl/estadisticas/economia/agricultura-agroindustria-y-pesca
# Aquí creo que es mas fácil que ellos revisen y vean cual es la información que les puede ser o solicitar BD filtrados
# al ine mediante solicitud y obtener la data (también lo podemos hacer nosotros pero no sé, deberiamos discutirlo con Carlos)
# 
